### Configurar advertencias

Para evitar que salgan advertencias que no nos interesan

In [84]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

### Importaciones

In [85]:
import pandas as pd
import sqlite3
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams.update(mpl.rcParamsDefault)
sns.set_context('notebook')
sns.set_theme()

## Preparar los datos

### Cargar datos

Vamos a crear una funcion para cargar los datos a un dataframe usando pandas. Un proceso extremadamente sencillo

In [86]:
def load_data(url: str) -> pd.DataFrame | None:
    try:
        return pd.read_excel(url)
    except Exception as err:
        print(f"Error loading data: {err}")
        return

Si la funcion logra cargar los datos, seran retornados en un Dataframe, si no, imprime el error y retorna None

### Limpiar datos

Creamos una funcion para limpiar los datos que recibe el dataframe cargado

In [87]:
def cleaning_data(df: pd.DataFrame):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    subset = ["restaurant_name", "order_time", "delivery_time", "order_hour"]
    df.dropna(subset=subset, inplace=True)
    df.drop_duplicates(inplace=True)
    df["order_time"] = pd.to_datetime(df["order_time"])
    df["delivery_time"] = pd.to_datetime(df["delivery_time"])

Primero se encarga de convertir todas los nombres de las columnas a minusculas y reemplaza sus espacios por ( _ ) para que sea mas facil acceder luego a ellas. Despues verificamos que las filas escenciales con las que vamos a trabajar no sean nulas, de ser asi las elimina, luego eliminamos las filas duplicadas y por ultimo todas las columnas de tipo fecha las transformamos a el tipo de dato correcto en pandas, para compatibilidad y mejor organizacion

### Guardar datos

Este paso consiste en guardar estos datos en una base SQL para su posterior uso (yo usare SQLite3). Esta funcion recibe el dataframe cargado y limpiado y una ruta donde guardar el archivo ".db" donde se almacenara la DB

In [88]:
def save_data(df: pd.DataFrame, db_url: str):
    with sqlite3.connect(db_url) as conn:
        try:
            df.to_sql("pizza_sell_data", conn, if_exists="replace", index=False)
        except Exception as err:
            print(f"Error saving data: {err}")
            return

Simplemente intenta conectarse a la base de datos pasada como argumento (que ya debe estar creada) e intenta convertir el dataframe a una tabla SQL, en caso de error pues imprime el error y retorna None

## Generacion de graficos

Vamos a crear una funcion que nos sirva para generar los graficos que necesitemos en todo el proyecto de manera simple

In [89]:
def generate_plot(data: pd.DataFrame, title: str, x: str, y: str, kind: str, style: str, palette: str, labels: tuple[str, str], path: str):
    try:
        plt.figure(figsize=(11, 9))
        sns.set_style(style)
        if kind == "bar":
            sns.barplot(data, x=x, y=y, palette=palette)
            
        plt.title(title)
        plt.xlabel(labels[0])
        plt.ylabel(labels[1])
        plt.xticks(rotation=90)
        plt.savefig(os.path.join(path, "graph.png"), bbox_inches='tight')
        
    except Exception as err:
        print(f"Error generating plot: {err}")
        return

## ETL (Extract, Tranform, Load)

In [90]:
data_url = "Enhanced_pizza_sell_data_2024-25.xlsx"
db_url = "database.db"

df = load_data(data_url)
cleaning_data(df)
save_data(df, db_url)

## Reportes

La manera en la que se van a organizar seria:

- Consulta y su ejecucion
- Asignar un titulo
- Crear y verificar la ruta donde se guardaran los datos
- Crear un excel y un grafico del reporte generado en la ruta especificada

### Total de pedidos por restaurante

In [91]:
with sqlite3.connect(db_url) as conn:
    query = """
    SELECT restaurant_name, COUNT(*) as orders
    FROM pizza_sell_data
    GROUP BY restaurant_name
    ORDER BY orders DESC
    """
    report = pd.read_sql_query(query, conn)
    title = "Pedidos por restaurante"
    
    path = os.path.join("reports", title.replace(' ', '-'))
    if not os.path.exists(path): os.mkdir(path)
    
    report.to_excel(os.path.join(path, "Data.xlsx"), index=False)
    
    generate_plot(report, title, "restaurant_name", "orders", 
                  "bar", "darkgrid", "deep", ("Restaurantes", "Pedidos"), path)